<a href="https://colab.research.google.com/github/Zheng-Ao/Colab-Notebooks/blob/main/P0_v05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# library install & cuda device

In [2]:
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"

1.12.0+cu113
11.3


In [3]:
!pip install transformers
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
!nvidia-smi
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import pandas as pd
raw_data_path = "drive/MyDrive/P0/T10K.csv"

pd.read_csv(raw_data_path)

Sat Jul 30 08:37:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

,Unnamed: 0,patent_number,patent_date,patent_num_cited_by_us_patents,patent_title,patent_abstract,patent_num_claims,patent_num_us_patent_citations,inventors,assignees,IPCs,cited_patents
0,0,3930276,1976-01-06,4,Wheel spinning and vehicle conveying apparatus...,An automobile conveyor for use in conjunction ...,12,2,"[{'inventor_sequence': '0', 'inventor_key_id':...","[{'assignee_sequence': '0', 'assignee_key_id':...",[{'ipc_sequence': '0'}],"[{'cited_patent_number': '3037223', 'cited_pat..."
1,1,3930279,1976-01-06,5,Rubber windshield wiper blades having increase...,A rubber windshield wiper blade is clamped to ...,2,3,"[{'inventor_sequence': '0', 'inventor_key_id':...","[{'assignee_sequence': None, 'assignee_key_id'...",[{'ipc_sequence': '0'}],"[{'cited_patent_number': '2140453', 'cited_pat..."
2,2,3930323,1976-01-06,17,Chain tensioning mechanism for scraper elevato...,A tensioning mechanism for the chain of a scra...,2,4,"[{'inventor_sequence': '0', 'inventor_key_id':...","[{'assignee_sequence': '0', 'assignee_key_id':...","[{'ipc_sequence': '0'}, {'ipc_sequence': '1'}]","[{'cited_patent_number': '372157', 'cited_pate..."
3,3,3930526,1976-01-06,14,Pneumatic tire and wheel assemblies,A pneumatic tire and wheel assembly comprises ...,21,7,"[{'inventor_sequence': '0', 'inventor_key_id':...","[{'assignee_sequence': '0', 'assignee_key_id':...",[{'ipc_sequence': '0'}],"[{'cited_patent_number': '1359461', 'cited_pat..."
4,4,3930527,1976-01-06,19,Tire and wheel assembly,A wheel having a pair of spaced-apart seats fo...,8,13,"[{'inventor_sequence': '0', 'inventor_key_id':...","[{'assignee_sequence': '0', 'assignee_key_id':...","[{'ipc_sequence': '0'}, {'ipc_sequence': '1'},...","[{'cited_patent_number': '1921772', 'cited_pat..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,4358129,1982-11-09,1,Tractor with a built-on underframe for a tilli...,The back part of the underframe is fastened to...,2,4,"[{'inventor_sequence': '0', 'inventor_key_id':...","[{'assignee_sequence': None, 'assignee_key_id'...",[{'ipc_sequence': '0'}],"[{'cited_patent_number': '2707643', 'cited_pat..."
9996,9996,4358133,1982-11-09,32,Adjustable width trailer,A trailer frame having fixed wheel track is pr...,6,6,"[{'inventor_sequence': '0', 'inventor_key_id':...","[{'assignee_sequence': None, 'assignee_key_id'...",[{'ipc_sequence': '0'}],"[{'cited_patent_number': '3239274', 'cited_pat..."
9997,9997,4358135,1982-11-09,22,Connector for igniting circuit of priming device,A connector provided in an igniting circuit fo...,9,7,"[{'inventor_sequence': '0', 'inventor_key_id':...","[{'assignee_sequence': '0', 'assignee_key_id':...","[{'ipc_sequence': '0'}, {'ipc_sequence': '1'}]","[{'cited_patent_number': '3509297', 'cited_pat..."
9998,9998,4358136,1982-11-09,32,Energy absorbing device for use with vehicular...,An energy absorbing device for use with a vehi...,7,7,"[{'inventor_sequence': '0', 'inventor_key_id':...","[{'assignee_sequence': '0', 'assignee_key_id':...",[{'ipc_sequence': '0'}],"[{'cited_patent_number': '3547468', 'cited_pat..."


# utils.py

## Txt2Vec Matrix, PCA

In [4]:
raw_data_path = "drive/MyDrive/P0/T10K.csv"

# 构建出txt_vecs，作为Dataset的第二个数据来源
import pandas as pd
from tqdm import tqdm

from transformers import DistilBertTokenizer, DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")      
nlp_model = DistilBertModel.from_pretrained("distilbert-base-uncased").to(device)

'''加载BERT这种大模型很耗时，因此在整个流程中应当让上面两行代码只执行一次。'''


df = pd.read_csv(raw_data_path)

for i in tqdm(range(10000)):
    ttl = df["patent_title"].values[i]
    inputs = tokenizer(ttl, return_tensors="pt").to(device)
    outputs = nlp_model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    cls_vec = last_hidden_states[:,0,:].clone().detach()
    if i == 0:
        txt_vecs = cls_vec
    else:
        txt_vecs = torch.cat([txt_vecs, cls_vec],dim=0)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 10000/10000 [01:09<00:00, 143.08it/s]


In [ ]:
# txt_vecs.shape

In [5]:
Mat = txt_vecs.cpu().numpy()

In [ ]:
# Mat.shape

In [6]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [7]:
# txt_pca = PCA(n_components=0.9, svd_solver = 'full')
txt_pca = PCA(n_components = 5)

In [ ]:
# txt_pca.fit(Mat)
# var = txt_pca.explained_variance_ratio_
# len(var), var.sum()

In [8]:
new_Mat = txt_pca.fit_transform(Mat)

## utils

In [9]:
from torch.utils.data import Dataset, Subset, ConcatDataset, DataLoader
from torch import nn

import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn import metrics

import random

from datetime import datetime
# target transform, 0:neg, 1:pos
def LabelCorePa(ref, isd):
    now = datetime.strptime("2022-01-01", "%Y-%m-%d").year
    years = now - datetime.strptime(isd, "%Y-%m-%d").year
    score = ref/years       
    label = int((score>0.5))
    return label

# Dataset
class PatDataset(Dataset):
    def __init__(self, raw_data_path, txt_vecs, transform = None, target_transform = LabelCorePa):
        self.raw_data = pd.read_csv(raw_data_path)
        self.txt_vecs = txt_vecs
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.raw_data)

    def __getitem__(self, idx):
        # Y          
        ref = self.raw_data.at[idx, "patent_num_cited_by_us_patents"]
        isd = self.raw_data.at[idx, "patent_date"]
        if self.target_transform:
            label = self.target_transform(ref, isd)
            label = torch.tensor(label,dtype=torch.long)

        # X
        # INDEXs
        num_claims = self.raw_data.at[idx, "patent_num_claims"]
        num_b_cits = self.raw_data.at[idx, "patent_num_us_patent_citations"]
        inventors = self.raw_data.at[idx, "inventors"]
        num_inventors = len(eval(inventors))
        assignees = self.raw_data.at[idx, "assignees"]
        if assignees == "[{'assignee_sequence': None, 'assignee_key_id': None}]":
            num_assignees = 0
        else:
            num_assignees = len(eval(assignees))
        IPCs = self.raw_data.at[idx, "IPCs"]
        num_ipcs = len(eval(IPCs))
        indexs = torch.tensor([num_claims, num_b_cits, num_inventors, num_assignees, num_ipcs], dtype=torch.float32).to(device)
        # TXT
        txt_vec = torch.tensor(self.txt_vecs[idx], dtype = torch.float32)
        txt_vec = txt_vec.to(device)

        patent = torch.cat([indexs, txt_vec])
        
        return patent, label

# Models
class SimpleNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNet, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.BatchNorm1d(input_size),
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_size, hidden_size)
        self.conv2 = GCNConv(hidden_size, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        logits = self.conv2(x, edge_index)                   # (num_nodes, 2)
        return logits                  # (num_nodes, 2)



# Train
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):

        # batch: 第几个batch；X: 包含batch_size个feature vec.
        X = X.to(device)
        y = y.to(device)

        output = model(X).to(device)
        loss = loss_fn(output, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 20 == 0:                                                     # train_size/batch_size = 100, 每20个batch输出一次结果，共输出5次。
            loss, current = loss.item(), batch * len(X)                         
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# Test
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X = X.to(device)
            y = y.to(device)
            output = model(X)
            # test_loss += loss_fn(output, y).item()
            # correct += (output.argmax(1) == y).type(torch.float).sum().item()
            
            pred = output.argmax(1).cpu()
            y = y.cpu()
            if batch == 0:
                Pred = pred
                Y = y
            else:
                Pred = torch.cat((Pred, pred), dim = 0)
                Y = torch.cat((Y, y), dim=0)

    C_Mat = metrics.confusion_matrix(Y, Pred)
    accuracy = metrics.accuracy_score(Y,Pred)
    f1 = metrics.f1_score(Y, Pred)
    recall = metrics.recall_score(Y, Pred)
    precision = metrics.precision_score(Y, Pred)
    print(C_Mat)
    print(f"acc:{accuracy:.4f}, f1:{f1:.4f}, recall:{recall:.4f}, prec:{precision:.4f}")

    # test_loss /= num_batches
    # correct /= size
    # print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# config.py

In [10]:
# HyperParams----Config.py
hidden_size = 32
output_size = 2

learning_rate = 1e-3
weight_decay = 5e-4
batch_size = 64                                                                 # test_size/batch_size = 25, 25 batches.
epochs = 10

num_train = 8000
num_test = 2000

# Main

## Net

In [11]:
new_Mat.shape

(10000, 5)

In [33]:
# PIPLINE
# DATA TO FIT A MODEL
dataset = PatDataset(raw_data_path=raw_data_path, txt_vecs = new_Mat)
training_indices = [i for i in range(num_train)]
test_indices = [i for i in range(num_train,num_train+num_test)]
training_data_all = Subset(dataset, training_indices)
test_data = Subset(dataset, test_indices)

# Make pos:neg in training set 1:1
pos_indices = []
neg_indices = []
for i in range(num_train):
    if training_data_all[i][1] == 1:
        pos_indices.append(i)
    else:
        neg_indices.append(i)
num_pos = len(pos_indices)
num_neg = len(neg_indices)
print(f"{num_neg}:{num_pos} = {num_neg/num_pos}")
neg_indices_sample = np.random.choice(neg_indices, num_pos, replace = False)	 
training_data_pos = Subset(training_data_all, pos_indices)
training_data_neg = Subset(training_data_all, neg_indices_sample)
training_data = ConcatDataset([training_data_pos, training_data_neg])
print("Training sample number:",len(training_data))

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

6633:1367 = 4.852231163130944
Training sample number: 2734
Feature batch shape: torch.Size([64, 10])
Labels batch shape: torch.Size([64])


In [ ]:
# FIT A MODEL
# PyTorch的逻辑是先初始化（喂超参），再进行函数计算（喂输入）
input_size = len(dataset[0][0])
model = SimpleNet(input_size, hidden_size, output_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(train_dataloader, model, loss_fn)                                 # performance on training data
    test_loop(test_dataloader, model, loss_fn)                                  # performance on test data
print("Done!")

## PyG GNN Net

In [13]:
# edge_index, edge_attr
df = pd.read_csv(raw_data_path)

adj_ls = []
edges = []

for i in tqdm(range(10000)):
    id = df.values[i][0]
    b_cits = eval(df["cited_patents"].values[i])
    for b_cit in b_cits:
        # 是否在专利数据库中
        if b_cit["cited_patent_date"]:
            # 是否在本数据集中
            if b_cit["cited_patent_number"] in df["patent_number"].values:
                # print(i)
                cit_id = df[df["patent_number"].values == b_cit["cited_patent_number"]].values[0][0]
                adj_ls.append([id, cit_id])
                date = df["patent_date"].values[id]
                cit_date = df["patent_date"].values[cit_id]
                date1 = datetime.strptime(date, "%Y-%m-%d")
                date2 = datetime.strptime(cit_date, "%Y-%m-%d")
                dist = date1-date2
                dist = 365/dist.days
                edges.append(dist)

100%|██████████| 10000/10000 [00:25<00:00, 384.99it/s]


In [32]:
# df[df["patent_number"].values == '3930276'].values[0][0]
# "3930276" in df["patent_number"].values
# edges
ls = [i for i in range(10)]

ls_1 = np.random.choice(ls,5, replace = False)
ls_1

array([1, 2, 4, 0, 9])

In [14]:
# data.x, data.y
for i in range(10000):
    if i == 0:
        x = dataset[i][0].unsqueeze(0)
        y = dataset[i][1].unsqueeze(0)
    else:
        x = torch.cat([x, dataset[i][0].unsqueeze(0)])
        y = torch.cat([y, dataset[i][1].unsqueeze(0)])

# 

In [ ]:
x.shape, x.dtype, y.shape, y.dtype

In [34]:
print(len(pos_indices),len(neg_indices_sample))
print(neg_indices_sample)
# len = 10
# indices = [1,2,4,5]
 
# ls1 = [False]*10
# t = torch.tensor(ls1)
# t[indices] = True
# t

# for i in range(len(pos_indices)):
#     if pos_indices[i] == neg_indices_sample[i]:
#         print(i)

ls = [False]*10000

# ls[pos_indices] = True

ls_t = torch.tensor(ls, dtype = bool)
# ls_t[pos_indices] = True
ls_t[neg_indices_sample] = True
ls_t.sum()


1367 1367
[7048 7989 1804 ... 2116 6281 2491]


tensor(1367)

In [35]:
from torch_geometric.data import Data

# num_nodes, num_node_features
x = torch.tensor(x, dtype=torch.float32)

# num_nodes, 1
y = torch.tensor(y, dtype=torch.long)

# 2, num_edges
edge_index = torch.tensor(adj_ls, dtype=torch.long)

# num_edges, num_edge_features
edge_attr = torch.tensor(edges, dtype=torch.float32)

# train-test split
train_ls = [False]*num_train + [False]*num_test
train_mask = torch.tensor(train_ls, dtype=bool)
train_mask[pos_indices] = True
train_mask[neg_indices_sample] = True
test_ls = [False]*num_train + [True]*num_test
test_mask = torch.tensor(test_ls, dtype=bool)


data = Data(x=x, y=y, edge_index = edge_index.t().contiguous(), edge_attr=edge_attr, train_mask = train_mask, test_mask=test_mask)
data = data.to(device)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


In [36]:
print(f'Number of nodes: {data.num_nodes}') # 节点数量
print(f'Number of edges: {data.num_edges}') # 边数量
print(f'Number of node features: {data.num_node_features}') # 节点属性的维度
print(f'Number of node features: {data.num_features}') # 同样是节点属性的维度
print(f'Number of edge features: {data.num_edge_features}') # 边属性的维度
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}') # 平均节点度
print(f'if edge indices are ordered and do not contain duplicate entries.: {data.is_coalesced()}') # 是否边是有序的同时不含有重复的边
print(f'Number of training nodes: {data.train_mask.sum()}') # 用作训练集的节点
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}') # 用作训练集的节点的数量
print(f'Contains isolated nodes: {data.has_isolated_nodes()}') # 此图是否包含孤立的节点
print(f'Contains self-loops: {data.has_self_loops()}')  # 此图是否包含自环的边
print(f'Is undirected: {data.is_undirected()}')  # 此图是否是无向图

Number of nodes: 10000
Number of edges: 5561
Number of node features: 10
Number of node features: 10
Number of edge features: 1
Average node degree: 0.56
if edge indices are ordered and do not contain duplicate entries.: False
Number of training nodes: 2734
Training node label rate: 0.27
Contains isolated nodes: True
Contains self-loops: False
Is undirected: False


In [37]:
input_size = len(dataset[0][0])

model = GNN(input_size, hidden_size, output_size).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_fn = nn.CrossEntropyLoss()

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [38]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')
Pred = pred[data.test_mask].cpu()
Y = data.y[data.test_mask].cpu()
C_Mat = metrics.confusion_matrix(Y, Pred)
accuracy = metrics.accuracy_score(Y,Pred)
f1 = metrics.f1_score(Y, Pred)
recall = metrics.recall_score(Y, Pred)
precision = metrics.precision_score(Y, Pred)
print(C_Mat)
print(f"acc:{accuracy:.4f}, f1:{f1:.4f}, recall:{recall:.4f}, prec:{precision:.4f}")


Accuracy: 0.6315
[[1052  507]
 [ 230  211]]
acc:0.6315, f1:0.3641, recall:0.4785, prec:0.2939


# Save&Load Model

In [ ]:
# 保存模型权重至当前文件夹
torch.save(model.state_dict(), 'model_weights.pth')                             

In [ ]:
model = SimpleNet(input_size, hidden_size, output_size)                         # 需要是同一个模型
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

'''
be sure to call model.eval() method before inferencing to set the dropout and batch normalization layers to evaluation mode. 
Failing to do this will yield inconsistent inference results.
'''

In [ ]:
# 直接保存/加载整个模型
torch.save(model, 'model.pth')

In [ ]:
model = torch.load('model.pth')

# Discoveries

事实证明，只用ttl几乎相当于没有给模型提供有用信息，模型倾向于只预测其中一类。